In [1]:
import collections
collections.Iterable = collections.abc.Iterable
import time
import sys
import urx
import numpy as np
import math
import threading
import io
import socket
import struct
from PIL import Image
import cv2
import numpy
import pupil_apriltags as apriltag
import time
from scipy.spatial.transform import Rotation as R
import transforms3d

In [2]:
tag = False
tag_pose = []

In [3]:
def coord_receiver(connection,detector,camera_params,tag_size,robot,goal,home,accuracy):
    global tag,tag_pose,threads_active
    print("coord_reciever thread started")
    i = 0
    while threads_active:
        i += 1
        image_len = struct.unpack(
            '<L', connection.read(struct.calcsize('<L')))[0]
        # print('I am here')
        if not image_len:
            break
        image_stream = io.BytesIO()
        image_stream.write(connection.read(image_len))
        image_stream.seek(0)
        image = Image.open(image_stream)
        opencvImage = cv2.cvtColor(numpy.array(image), cv2.COLOR_RGB2BGR)
        gray = cv2.cvtColor(opencvImage, cv2.COLOR_BGR2GRAY)
        result = detector.detect(
            gray, estimate_tag_pose=True, camera_params=camera_params, tag_size=tag_size)
        cv2.imshow('im',opencvImage)

        current_rob_pose = robot.getl()
        diff = -np.array(current_rob_pose[:3])+np.array(goal[:3])
        v = diff/np.max(np.abs(diff))*0.01
        robot.speedl((v[0],v[1],v[2],0,0,0),0.05,1)
        distance = np.linalg.norm(diff)

        if distance <=accuracy:
            robot.speedl((0,0,0,0,0,0),0.05,1)
            threads_active = False

        if result and np.linalg.norm(result[0].pose_t) < 0.35:
            print('COLLISION')
            robot.speedl((0,0,0,0,0,0),0.05,1)
            robot.movel((home[0],home[1],home[2],home[3],home[4],home[5]),0.01,0.01)
            threads_active = False
        key = cv2.waitKey(10) & 0xFF

        if key == ord('q'):
            threads_active = False

        


In [4]:
camera_params = (506.19083684, 508.36108854,
                 317.93111342, 243.12403806)
tag_size = 0.0375

UDP_IP = "127.0.0.1"
UDP_PORT = 5065
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_socket = socket.socket()
server_socket.bind(('0.0.0.0', 8000))
server_socket.listen(0)
connection = server_socket.accept()[0].makefile('rb')

accuracy = 0.001

detector = apriltag.Detector(
    families='tag36h11',
    nthreads=10,
    quad_decimate=0.5,
    quad_sigma=0,
    refine_edges=1,
    decode_sharpening=0.6,
    debug=0)



In [5]:
rob = urx.Robot('192.168.88.60')
home = [-0.297,
 -0.142,
 0.49,
 0.11,
 -1.52,
 0.19]
goal = [-0.43,
 -0.142,
 0.49,
 0.11,
 -1.52,
 0.19]

In [6]:
rob.getl()

[-0.2969454121424089,
 -0.1419955301769424,
 0.49007065595804217,
 0.10998178043588865,
 -1.519838379616969,
 0.18988991748146059]

In [7]:
threads_active=True
coord_rec_thread = threading.Thread(
target=coord_receiver,
daemon=True,
args=(connection,detector,camera_params,tag_size,rob,home,goal,accuracy)
)
coord_rec_thread.start()

coord_reciever thread started


In [ ]:
coord_rec_thread.join()

In [8]:

threads_active=True
coord_rec_thread = threading.Thread(
target=coord_receiver,
daemon=True,
args=(connection,detector,camera_params,tag_size,rob,goal,home,accuracy)
)
coord_rec_thread.start()

coord_reciever thread started


In [8]:
threads_active=False
connection.close()


In [19]:
rob.movel(goal,0.05,0.01)

<Transform:
<Orientation: 
array([[ 0.03965884, -0.19215043, -0.98056381],
       [ 0.05521701,  0.98025778, -0.18985721],
       [ 0.99768645, -0.04661429,  0.04948586]])>
<Vector: (-0.41007, -0.14197, 0.48994)>
>

In [9]:
rob.movel(home,0.05,0.01)

<Transform:
<Orientation: 
array([[ 0.03996381, -0.19206053, -0.98056904],
       [ 0.05519772,  0.98027884, -0.18975406],
       [ 0.99767535, -0.04654188,  0.04977698]])>
<Vector: (-0.29702, -0.14199, 0.49000)>
>

In [3]:
cos = -np.sqrt(2)/2
sin = np.sqrt(2)/2
theta = np.arctan2(sin,cos)
theta = np.rad2deg(theta)
theta = int(theta)
print(theta)

2
